<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/TensorFlow%20Overview/Quickstart%20for%20Experts/TensorFlow_2_0_Quickstart_for_Experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 85kB/s 
     |████████████████████████████████| 450kB 31.7MB/s 
     |████████████████████████████████| 3.8MB 28.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
from __future__ import absolute_import, print_function, unicode_literals, division
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets.mnist import load_data

In [0]:
(X_train, y_train), (X_test, y_test) = load_data()
X_train, X_test = X_train/255.0, X_test/255.0

X_train = X_train[..., None]
X_test = X_test[..., None]

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [22]:
print(train_ds.element_spec)
print(test_ds.element_spec)

(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))
(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))


In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1 = Conv2D(32,3,activation=tf.nn.relu)
        self.flatten = Flatten()
        self.dense1 = Dense(128,activation=tf.nn.relu)
        self.dense2 = Dense(10,activation=tf.nn.softmax)
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

model = MyModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [0]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_accuracy.update_state(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

In [37]:
EPOCHS = 5
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    
    for images, labels in test_ds:
        test_step(images, labels)

    template = "Epoch {}, Loss {}, Accuracy {}, Test Loss {}, Test Accuracy {}"
    print(template.format(epoch+1, 
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss 0.06489448994398117, Accuracy 97.9927749633789, Test Loss 0.051087651401758194, Test Accuracy 98.28333282470703
Epoch 2, Loss 0.01323071587830782, Accuracy 99.56832885742188, Test Loss 0.056977882981300354, Test Accuracy 98.44999694824219
Epoch 3, Loss 0.008983412757515907, Accuracy 99.68499755859375, Test Loss 0.0543401874601841, Test Accuracy 98.3499984741211
Epoch 4, Loss 0.006586488801985979, Accuracy 99.7683334350586, Test Loss 0.06609255075454712, Test Accuracy 98.43999481201172
Epoch 5, Loss 0.0056251781061291695, Accuracy 99.80833435058594, Test Loss 0.08277901262044907, Test Accuracy 98.38999938964844
